Installing dependencies

In [9]:
%%script echo skipping
# %pip install binance-connector
%pip install tradingview-ta
%pip install pandas
%pip install python-dotenv
%pip install python-binance

skipping


Trying out TradingView API

In [10]:
%%script echo skipping
from tradingview_ta import TA_Handler, Interval, Exchange

ADA = TA_Handler(
    symbol="ADAUSDT",
    screener="crypto",
    exchange="BINANCE",
    interval=Interval.INTERVAL_1_DAY,
    # proxies={'http': 'http://example.com:8080'} # Uncomment to enable proxy (replace the URL).
)
print(ADA.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

skipping


In [11]:
import json
import os
from datetime import datetime

import numpy as np
import pandas as pd
import requests
from binance.client import Client

In [12]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_PASSWORD']
ta_api = os.environ['taapi_key']


# client = Client(api_key, api_secret)
# print('Using Binance TestNet Server')
# print('')
# print(binance_client.get_account())
# client = Client(base_url='https://testnet.binance.vision')
# print(client.time())
# print(client.account())

# from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
client = Client(api_key, api_secret, testnet=True)
# client.ping()
# client.get_server_time()
# info = client.get_symbol_info('ETHUSDT')
# print(info)

In [13]:
def get_data_frame(symbol):
    # starttime = '6 months ago UTC'
    starttime = '1 Jan, 2017'
    interval = '1h'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])

    return df

get_data_frame('ETHUSDT')

,date,open,high,low,close
0,1667394000000,1556.31000000,1557.38000000,1414.70000000,1553.45000000
1,1667397600000,1553.25000000,1572.20000000,1523.86000000,1560.21000000
2,1667401200000,1560.22000000,1561.57000000,1481.81000000,1558.54000000
3,1667404800000,1558.51000000,1574.85000000,1550.09000000,1571.23000000
4,1667408400000,1570.95000000,1577.10000000,1550.91000000,1558.57000000
...,...,...,...,...,...
117,1667815200000,1563.88000000,1584.32000000,1536.01000000,1578.53000000
118,1667818800000,1578.29000000,3421.00000000,1393.13000000,1579.96000000
119,1667822400000,1579.85000000,1584.98000000,1576.49000000,1584.57000000
120,1667826000000,1584.51000000,1589.53000000,1575.41000000,1579.34000000


In [14]:
def macd_logic(symbol):

    symbol_df = get_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['200_ema'] < symbol_df['close'])), 1, 0)

    return symbol_df

macd_logic('ETHUSDT')

,date,open,high,low,close,macd,signal,200_ema,Trigger
0,2022-11-02 13:00:00,1556.31,1557.38,1414.70,1553.45,0.000000,0.000000,1553.450000,0
1,2022-11-02 14:00:00,1553.25,1572.20,1523.86,1560.21,0.539259,0.107852,1553.517264,1
2,2022-11-02 15:00:00,1560.22,1561.57,1481.81,1558.54,0.822391,0.250760,1553.567241,1
3,2022-11-02 16:00:00,1558.51,1574.85,1550.09,1571.23,2.047154,0.610038,1553.742990,1
4,2022-11-02 17:00:00,1570.95,1577.10,1550.91,1558.57,1.973482,0.882727,1553.791020,1
...,...,...,...,...,...,...,...,...,...
117,2022-11-07 10:00:00,1563.88,1584.32,1536.01,1578.53,-13.380135,-12.086644,1585.135100,0
118,2022-11-07 11:00:00,1578.29,3421.00,1393.13,1579.96,-12.333179,-12.135951,1585.083607,0
119,2022-11-07 12:00:00,1579.85,1584.98,1576.49,1584.57,-11.004615,-11.909684,1585.078496,0
120,2022-11-07 13:00:00,1584.51,1589.53,1575.41,1579.34,-10.255517,-11.578851,1585.021397,0


# BACKTESTING

Backtesting completed on 6 month historical data on 5m time frame

In [15]:
def get_six_month_data_frame(symbol):
    # starttime = '6 months ago UTC'
    starttime = '1 Jan, 2017'
    interval = '1h'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])
    return df

six_month_eth_df_raw = get_six_month_data_frame('ETHUSDT')

In [16]:
# url1 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1660629600000'
# url2 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1664229600000'
# url3 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1667829600000'

def get_bars(symbol, limit = '1000', interval = '1h'):
    df = pd.DataFrame()
    root_url = 'https://api.binance.com/api/v1/klines'
    end_times = ['1660629600000', '1664229600000', '1667829600000']
    for time in end_times:
        url = root_url + '?symbol=' + symbol + '&interval=' + interval + '&limit=' + limit + '&endTime=' + time
        data = json.loads(requests.get(url).text)
        for row in data:
            del row[5:]
        df = df.append(data)
    df.columns = ['date',
                'open', 'high', 'low', 'close']
    return df.reset_index()

eth_df_raw = get_bars('ETHUSDT')

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_49625/2673867202.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)


In [17]:
eth_df_raw

,index,date,open,high,low,close
0,0,1657033200000,1093.38000000,1095.08000000,1076.34000000,1092.95000000
1,1,1657036800000,1092.95000000,1103.50000000,1088.28000000,1095.60000000
2,2,1657040400000,1095.60000000,1107.41000000,1094.33000000,1102.68000000
3,3,1657044000000,1102.69000000,1128.00000000,1099.44000000,1127.51000000
4,4,1657047600000,1127.50000000,1152.75000000,1123.82000000,1146.53000000
...,...,...,...,...,...,...
2995,995,1667815200000,1564.10000000,1584.56000000,1563.88000000,1578.54000000
2996,996,1667818800000,1578.53000000,1582.02000000,1575.66000000,1580.00000000
2997,997,1667822400000,1579.99000000,1585.02000000,1576.48000000,1584.51000000
2998,998,1667826000000,1584.50000000,1589.73000000,1575.31000000,1579.30000000


In [18]:
def test_macd_logic(symbol):

    symbol_df = get_bars(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200

    # print(symbol_df.info())


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['close'] > symbol_df['200_ema']) & (symbol_df['macd'] < 0) & (symbol_df['signal'] > symbol_df['macd'].shift())), 1, 0)
    
    tr_list = [(symbol_df['high'] - symbol_df['low']), (symbol_df['low'] - symbol_df['close'].shift()), (symbol_df['high'] - symbol_df['close'].shift())]

    symbol_df['TR1'] = symbol_df['high'] - symbol_df['low']
    symbol_df['prev close'] = symbol_df['close'].shift()
    symbol_df['TR2'] = symbol_df['low'] - symbol_df['close'].shift()
    symbol_df['TR3'] = symbol_df['high'] - symbol_df['close'].shift()
    # symbol_df['TR LIST'] = symbol_df.apply(lambda row : [(row['high'] - row['low']), (row['low'] - row['close'].shift()), (row['high'] - row['close'].shift())])
    # symbol_df['TR LIST'] = symbol_df.apply(symbol_df)

    symbol_df = symbol_df.iloc[200:]

    triggered_df = symbol_df.loc[symbol_df['Trigger'] == 1]

    return symbol_df

test_df = test_macd_logic('ETHUSDT')

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_49625/2673867202.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)


In [19]:
test_df

,index,date,open,high,low,close,macd,signal,200_ema,Trigger,TR1,prev close,TR2,TR3
200,200,2022-07-13 23:00:00,1086.36,1119.18,1085.99,1115.00,6.999715,1.880093,1126.556008,0,33.19,1086.36,-0.37,32.82
201,201,2022-07-14 00:00:00,1115.01,1123.54,1109.47,1114.02,8.988532,3.301781,1126.431272,0,14.07,1115.00,-5.53,8.54
202,202,2022-07-14 01:00:00,1114.02,1127.19,1110.46,1113.41,10.395627,4.720550,1126.301707,0,16.73,1114.02,-3.56,13.17
203,203,2022-07-14 02:00:00,1113.40,1116.36,1109.00,1112.20,11.283060,6.033052,1126.161391,0,7.36,1113.41,-4.41,2.95
204,204,2022-07-14 03:00:00,1112.20,1119.35,1110.46,1112.71,11.890443,7.204530,1126.027547,0,8.89,1112.20,-1.74,7.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,995,2022-11-07 10:00:00,1564.10,1584.56,1563.88,1578.54,-13.407928,-12.128339,1580.258033,0,20.68,1564.10,-0.22,20.46
2996,996,2022-11-07 11:00:00,1578.53,1582.02,1575.66,1580.00,-12.353065,-12.173284,1580.255466,0,6.36,1578.54,-2.88,3.48
2997,997,2022-11-07 12:00:00,1579.99,1585.02,1576.48,1584.51,-11.026057,-11.943839,1580.297799,1,8.54,1580.00,-3.52,5.02
2998,998,2022-11-07 13:00:00,1584.50,1589.73,1575.31,1579.30,-10.276338,-11.610339,1580.287871,0,14.42,1584.51,-9.20,5.22


In [20]:
atr_indicator = "atr"
atr_endpoint = f"https://api.taapi.io/{atr_indicator}"

live_price_indicator = "price"
live_price_endpoint = f"https://api.taapi.io/{atr_indicator}"

stop_loss = 0
tp = 0
entry = 0

parameters = {
    'secret': ta_api,
    'exchange': 'binance',
    'symbol': 'ETH/USDT',
    'interval': '1H'
    } 

in_trade = False
tp1_hit = False

win = 0
loss = 0
profit = 0
losses = 0

atr_test = 16.74326363683623

for index, row in test_df.iterrows():
    
    if in_trade == False:
        if row['Trigger'] == 1:
            in_trade = True

            stop_loss = row['close'] - atr_test
            tp = row['close'] + atr_test
            entry = row['close']
            
    
    else:
        if row['close'] <= stop_loss:
            in_trade = False
            close = row['close']
            loss += 1
            losses += (entry - stop_loss)
            tp1_hit = False
            print(f'STOP LOSS: {stop_loss}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Losses: {losses}')
            print('')

        elif row['close'] >= tp and tp1_hit == False:
            # in_trade = False
            close = row['close']
            tp1_hit = True
            stop_loss = entry
            win += 1
            profit += ((tp - entry)/2)
            print(f'TP1: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')
            tp = (entry + (2*atr_test))
            
        elif row['close'] >= tp and tp1_hit == True:
            close = row['close']
            in_trade = False
            tp1_hit = False
            profit += ((tp - entry)/2)
            print(f'TP2: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')


        
print(f'wins: {win}, profit: ${profit}')
print(f'losses: {loss}, losses: ${losses}')
print(f'P/L: {profit - losses}')

TP1: 1522.863263636836, CLOSE: 1558.37, ENTRY: 1506.12
Profit: 8.371631818418109

TP2: 1539.6065272736723, CLOSE: 1543.61, ENTRY: 1506.12
Profit: 25.114895455254327

STOP LOSS: 1570.2767363631638, CLOSE: 1550.4, ENTRY: 1587.02
Losses: 16.743263636836218

TP1: 1558.5932636368361, CLOSE: 1565.09, ENTRY: 1541.85
Profit: 33.486527273672436

TP2: 1575.3365272736723, CLOSE: 1601.33, ENTRY: 1541.85
Profit: 50.229790910508655

STOP LOSS: 1699.5567363631637, CLOSE: 1684.69, ENTRY: 1716.3
Losses: 33.486527273672436

TP1: 1663.8732636368363, CLOSE: 1664.69, ENTRY: 1647.13
Profit: 58.601422728926764

TP2: 1680.6165272736725, CLOSE: 1713.43, ENTRY: 1647.13
Profit: 75.34468636576298

TP1: 1724.6532636368363, CLOSE: 1733.69, ENTRY: 1707.91
Profit: 83.71631818418109

TP2: 1741.3965272736725, CLOSE: 1773.28, ENTRY: 1707.91
Profit: 100.45958182101731

STOP LOSS: 1686.0167363631638, CLOSE: 1672.21, ENTRY: 1702.76
Losses: 50.229790910508655

STOP LOSS: 1983.4067363631639, CLOSE: 1983.09, ENTRY: 2000.15
Lo

In [21]:
eth_raw_df_2 = get_six_month_data_frame('ETHUSDT')
eth_raw_df_2

,date,open,high,low,close
0,1667394000000,1556.31000000,1557.38000000,1414.70000000,1553.45000000
1,1667397600000,1553.25000000,1572.20000000,1523.86000000,1560.21000000
2,1667401200000,1560.22000000,1561.57000000,1481.81000000,1558.54000000
3,1667404800000,1558.51000000,1574.85000000,1550.09000000,1571.23000000
4,1667408400000,1570.95000000,1577.10000000,1550.91000000,1558.57000000
...,...,...,...,...,...
117,1667815200000,1563.88000000,1584.32000000,1536.01000000,1578.53000000
118,1667818800000,1578.29000000,3421.00000000,1393.13000000,1579.96000000
119,1667822400000,1579.85000000,1584.98000000,1576.49000000,1584.57000000
120,1667826000000,1584.51000000,1589.53000000,1575.41000000,1579.34000000


In [22]:
def test_logic_2(raw_df):
    raw_df['open'] = pd.to_numeric(raw_df['open'])
    raw_df['high'] = pd.to_numeric(raw_df['high'])
    raw_df['low'] = pd.to_numeric(raw_df['low'])
    raw_df['close'] = pd.to_numeric(raw_df['close'])
    raw_df['date'] = pd.to_datetime(raw_df['date'], unit = 'ms')


    short_ema = raw_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = raw_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = raw_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    raw_df['macd'] = macd
    raw_df['signal'] = signal
    raw_df['200_ema'] = ema_200

    raw_df['TR1'] = abs(raw_df['high'] - raw_df['low'])
    raw_df['TR2'] = abs(raw_df['low'] - raw_df['close'].shift())
    raw_df['TR3'] = abs(raw_df['high'] - raw_df['close'].shift())

    raw_df['ATR'] = raw_df.apply(lambda x: row_check(x.TR1, x.TR2, x.TR3), axis=1)

    raw_df = raw_df.iloc[200:]

    raw_df = raw_df.drop(['TR1', 'TR2', 'TR3'], axis=1)

    raw_df['Trigger'] = np.where(((raw_df['macd'] > raw_df['signal']) & (raw_df['close'] > raw_df['200_ema']) & (raw_df['macd'] < 0) & (raw_df['signal'] > raw_df['macd'].shift())), 1, 0)

    return raw_df

def row_check(TR1, TR2, TR3):
    if TR1 >= TR2 and TR1 >= TR3:
        ATR = TR1
    elif TR2 >= TR1 and TR2 >= TR3:
        ATR = TR2
    else:
        ATR = TR3
    
    return ATR

In [23]:
# tinkered_df = test_logic_2(eth_raw_df_2)
tinkered_df = test_logic_2(get_bars('ETHUSDT'))

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_49625/2673867202.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)


In [24]:
1504771200000
print(pd.to_datetime(1504771200000, unit = 'ms'))

2017-09-07 08:00:00


In [25]:
tinkered_df

,index,date,open,high,low,close,macd,signal,200_ema,ATR,Trigger
200,200,2022-07-13 23:00:00,1086.36,1119.18,1085.99,1115.00,6.999715,1.880093,1126.556008,33.19,0
201,201,2022-07-14 00:00:00,1115.01,1123.54,1109.47,1114.02,8.988532,3.301781,1126.431272,14.07,0
202,202,2022-07-14 01:00:00,1114.02,1127.19,1110.46,1113.41,10.395627,4.720550,1126.301707,16.73,0
203,203,2022-07-14 02:00:00,1113.40,1116.36,1109.00,1112.20,11.283060,6.033052,1126.161391,7.36,0
204,204,2022-07-14 03:00:00,1112.20,1119.35,1110.46,1112.71,11.890443,7.204530,1126.027547,8.89,0
...,...,...,...,...,...,...,...,...,...,...,...
2995,995,2022-11-07 10:00:00,1564.10,1584.56,1563.88,1578.54,-13.407928,-12.128339,1580.258033,20.68,0
2996,996,2022-11-07 11:00:00,1578.53,1582.02,1575.66,1580.00,-12.353065,-12.173284,1580.255466,6.36,0
2997,997,2022-11-07 12:00:00,1579.99,1585.02,1576.48,1584.51,-11.026057,-11.943839,1580.297799,8.54,1
2998,998,2022-11-07 13:00:00,1584.50,1589.73,1575.31,1579.30,-10.276338,-11.610339,1580.287871,14.42,0


In [30]:
stop_loss = 0
tp = 0
entry = 0

in_trade = False
tp1_hit = False

win = 0
loss = 0
profit = 0
losses = 0

for index, row in tinkered_df.iterrows():
    atr = 0
    
    if in_trade == False:
        if row['Trigger'] == 1:
            in_trade = True
            atr = row['ATR']
            stop_loss = row['close'] - atr
            tp = row['close'] + atr
            entry = row['close']
            print('')
            print('')
            print('OPENED TRADE')
            print(f'Bought 1 ETH @ {entry}')
            print(f'SL: {stop_loss}, TP1: {entry + atr}, TP2: {entry + 2*atr}')
            print('')
            
    
    else:
        if row['close'] <= stop_loss and tp1_hit == False:
            in_trade = False
            close = row['close']
            loss += 1
            losses += (entry - stop_loss)
            tp1_hit = False
            print(f'STOP LOSS: {stop_loss}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Losses: {losses}')
            print(f'Current P/L: ${profit-losses}')
            print('')

        if row['close'] <= stop_loss and tp1_hit == True:
            in_trade = False
            close = row['close']
            loss += 1
            losses += (entry - stop_loss)/2
            tp1_hit = False
            print(f'STOP LOSS: {stop_loss}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Losses: {losses}')
            print(f'Current P/L: ${profit-losses}')
            print('')

        elif row['close'] >= tp and tp1_hit == False:
            # in_trade = False
            close = row['close']
            tp1_hit = True
            stop_loss = entry
            win += 1
            profit += ((tp - entry)/2)
            print(f'TP1: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print(f'Current P/L: ${profit-losses}')
            print('')
            tp = (entry + (2*atr))
            
        elif row['close'] >= tp and tp1_hit == True:
            close = row['close']
            in_trade = False
            tp1_hit = False
            profit += ((tp - entry)/2)
            print(f'TP2: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print(f'Current P/L: ${profit-losses}')
            print('')


        
print(f'wins: {win}, profit: ${profit}')
print(f'losses: {loss}, losses: ${losses}')
print(f'P/L: ${profit - losses}')



OPENED TRADE
Bought 1 ETH @ 1506.12
SL: 1474.7699999999998, TP1: 1537.47, TP2: 1568.8200000000002

TP1: 1537.47, CLOSE: 1558.37, ENTRY: 1506.12
Profit: 15.675000000000068
Current P/L: $15.675000000000068

TP2: 1506.12, CLOSE: 1536.23, ENTRY: 1506.12
Profit: 15.675000000000068
Current P/L: $15.675000000000068



OPENED TRADE
Bought 1 ETH @ 1587.02
SL: 1554.79, TP1: 1619.25, TP2: 1651.48

STOP LOSS: 1554.79, CLOSE: 1550.4, ENTRY: 1587.02
Losses: 32.23000000000002
Current P/L: $-16.55499999999995



OPENED TRADE
Bought 1 ETH @ 1541.85
SL: 1507.1599999999999, TP1: 1576.54, TP2: 1611.23

TP1: 1576.54, CLOSE: 1601.33, ENTRY: 1541.85
Profit: 33.020000000000095
Current P/L: $0.7900000000000773

TP2: 1541.85, CLOSE: 1610.14, ENTRY: 1541.85
Profit: 33.020000000000095
Current P/L: $0.7900000000000773



OPENED TRADE
Bought 1 ETH @ 1716.3
SL: 1692.04, TP1: 1740.56, TP2: 1764.82

STOP LOSS: 1692.04, CLOSE: 1684.69, ENTRY: 1716.3
Losses: 56.49000000000001
Current P/L: $-23.469999999999914



OPENE

In [27]:
1660633200000 - 3600000

1660629600000

In [28]:
import requests

response = requests.get('https://cnft.tools/api/whitelist/autowl')

data = response.json()
pioneer_count = 0
wl_count = 0

for test in data:
    print('Pioneer')
    if test['role'] == '1018984763291340840':
        pioneer_count += 1
        print(test['role'])
        print(test['addr'])
        print('')

for test in data:
    print('WL')
    if test['role'] == '1019005107800637490':
        wl_count += 1
        print(test['role'])
        print(test['addr'])
        print('')

print(pioneer_count)
print(wl_count)

Pioneer
1018984763291340840
addr1qxrhwpe3gw75pp8ux7p7tshvac34dr0yhtrpj9g2jjgahccs98m6apttgmutjlekx08zk4uy6sk24v5apkvv9xsnznzqulfa49

Pioneer
1018984763291340840
addr1q8prqclry8mluma8sejt67rp6juryv9pmzep7ar4afp7jalguhf4g7srq59xeklh8xwatjd29qu4s7aam8p06ylatgvspx7cjn

Pioneer
1018984763291340840
addr1q9y2w3u7t0xpzmrue8846rd8sy9h5geu8h443rumkfxhlt545aest3dwktdnk76u6npcufryax4xd9ley3mhe5mxruls5zhvup

Pioneer
1018984763291340840
addr1qxcy9xwkdj2gsyg4xpzxqt6vssrr2tuvz964pfzlu3gfeus09tqs5rppqwtj25xjn5t6k9jmg5hrp8y0zsw064wu0kcs9f3hjl

Pioneer
1018984763291340840
addr1qxq4e7804g980q00ddpdkrhgh9el3h4uewczs2aef25ut764mya972xe80ufcreq42zhv6f2ghfkpa0jacsyzhvfs84ssa0x3j

Pioneer
1018984763291340840
addr1q9dk7r0kr5r8wfnm0vc6xzqe0nwj7fydn6pa34577edysrnk5elmd8nutfq6vl07epvj56my84rz455ctqnz6yv9m97qqfvlpz

Pioneer
1018984763291340840
addr1qx48v6xdgqyead9mym8w2hje7f4sk640jfdlhlamg25ranakevsa0fjalzt8udlw5ntcwfwz9jmx92g4wsnl7uxdwnvs5yv5g2

Pioneer
1018984763291340840
addr1qxyjemk5z2l9lwrs2xhvkq2slykltqa0krn2